## Column Description
<span style="font-size: 14px;">

**track_id**: The Spotify ID for the track

**artists**: The artists' names who performed the track. If there is more than one artist, they are separated by a ;

**album_name**: The album name in which the track appears

**track_name**: Name of the track

**popularity**: The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.

**duration_ms**: The track length in milliseconds

**explicit**: Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)

**danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable

**energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

**key**: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1

**loudness**: The overall loudness of a track in decibels (dB)

**mode**: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0

**speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks

**acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic

**instrumentalness**: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content

**liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live

**valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)

**tempo**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration

**time_signature**: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.

**track_genre**: The genre in which the track belongs
</span>

In [1]:
import helper as h
dir(h)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'descriptive_stats_extended',
 'is_int_or_float',
 'mean',
 'ms_to_minutes_seconds',
 'np',
 'pd']

In [4]:
arr = [2 , 3 , 4]
df = h.pd.DataFrame(arr)
print("my mean function:" + repr(h.mean(arr)) + " pandas mean function: " + repr(df.mean().iloc[0]))

my mean function:3.0 pandas mean function: np.float64(3.0)


In [2]:
df = h.pd.read_csv('dataset.csv', encoding='utf-8')#load the dataset as a pandas dataframe

df.dropna(inplace=True)#remove null values

In [5]:
#Calculate total number of individual entries
print( 'Total number of data entries = '  + repr(df.columns.size)  + ' * ' + repr(df['Unnamed: 0'].size) + ' = ' + repr(df.columns.size *  df['Unnamed: 0'].size))
print( 'Total number of data entries (excluding index column) = ' + repr(df.columns.size - 1)  + ' * ' + repr(df['Unnamed: 0'].size) + ' = ' + repr((df.columns.size - 1) * df['Unnamed: 0'].size))

Total number of data entries = 21 * 113999 = 2393979
Total number of data entries (excluding index column) = 20 * 113999 = 2279980


In [6]:
#this data frame is a summary of the number of values in each column and their data type
summarydf = h.pd.DataFrame(columns=['Column', 'Data type', 'Size', 'Number of unique Values'])

for loop_count, column in enumerate(df.columns):
    summarydf.loc[loop_count] = [column, df[column].dtype, df[column].size, df[column].nunique()]   #populate the dataframe with data from dataset 
    
summarydf.set_index('Column', inplace=True)

summarydf


,Data type,Size,Number of unique Values
Column,,,
Unnamed: 0,int64,113999,113999
track_id,object,113999,89740
artists,object,113999,31437
album_name,object,113999,46589
track_name,object,113999,73608
popularity,int64,113999,101
duration_ms,int64,113999,50696
explicit,bool,113999,2
danceability,float64,113999,1174


In [8]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,113999.0,56999.421925,32909.243463,0.000,28499.5000,56999.000000,85499.5000,113999.000
popularity,113999.0,33.238827,22.304959,0.000,17.0000,35.000000,50.0000,100.000
duration_ms,113999.0,228031.153387,107296.057716,8586.000,174066.0000,212906.000000,261506.0000,5237295.000
danceability,113999.0,0.566801,0.173543,0.000,0.4560,0.580000,0.6950,0.985
energy,113999.0,0.641383,0.251530,0.000,0.4720,0.685000,0.8540,1.000
key,113999.0,5.309126,3.559999,0.000,2.0000,5.000000,8.0000,11.000
loudness,113999.0,-8.258950,5.029357,-49.531,-10.0130,-7.004000,-5.0030,4.532
mode,113999.0,0.637558,0.480708,0.000,0.0000,1.000000,1.0000,1.000
speechiness,113999.0,0.084652,0.105733,0.000,0.0359,0.048900,0.0845,0.965
acousticness,113999.0,0.314907,0.332522,0.000,0.0169,0.169000,0.5975,0.996


In [11]:
h.descriptive_stats_extended(df)

,Column,Mean,Standard deviation,Variance,Min,Max,Range,Lower Quartile,Median,Upper Quartile
5,popularity,33.238827,22.304959,497.511199,0,100,100,17.0,35.0,50.0
6,duration_ms,228031.153387,107296.057716,11512444001.445007,8586,5237295,5228709,174066.0,212906.0,261506.0
7,duration minutes and seconds,3m and 48s,1m and 47s,191874m and 4s,0m and 8s,87m and 17s,87m and 8s,2m and 54s,3m and 32s,4m and 21s
8,danceability,0.566801,0.173543,0.030117,0.0,0.985,0.985,0.456,0.58,0.695
9,energy,0.641383,0.25153,0.063267,0.0,1.0,1.0,0.472,0.685,0.854
10,key,5.309126,3.559999,12.673594,0,11,11,2.0,5.0,8.0
11,loudness,-8.25895,5.029357,25.294436,-49.531,4.532,54.063,-10.013,-7.004,-5.003
12,mode,0.637558,0.480708,0.23108,0,1,1,0.0,1.0,1.0
13,speechiness,0.084652,0.105733,0.011179,0.0,0.965,0.965,0.0359,0.0489,0.0845
14,acousticness,0.314907,0.332522,0.110571,0.0,0.996,0.996,0.0169,0.169,0.5975


In [5]:
#TODO: group by genre
#    : compare popularity by genre
#    : compare most popular songs
#    : Analyse qualities which lead to greater popularity

In [13]:
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.express as px
import random

In [43]:
#this function creates a dash app layout for comparing
def compare_axes(data : h.pd.DataFrame, app : Dash): #enforce type of arguments to avoid ambiguity
    numerical_columns = data.select_dtypes(include=['number']).columns #this variable is to ensure only numerical columns can be selected for the graph axes
    categorical_columns = data.select_dtypes(include=['object', 'category', 'string']).columns
    using_index = False
    
    if len(categorical_columns) == 0:
        using_index = True
        categorical_columns = data['index'] = data.index  # Create a new column with the DataFrame index
    
    # Ensure the dataset passed in contains at least one categorical and two numerical columns
    if len(categorical_columns) < 1 or len(numerical_columns) < 2:
        raise ValueError("Data must contain at least one categorical (object) column and two numerical columns.")
    
    if not using_index:
        initial_category = random.choice(categorical_columns) if len(categorical_columns) > 0 else None #randomise the category column
 
    app.layout = html.Div([
        #dcc.Store(id='stored-data', data=data.to_dict()),
       # dcc.Store(id='previous-selected-values'),
       # dcc.Store(id='previous-selected-category'),
        html.Div([
            html.Label("Select Category:"),
            dcc.Dropdown(categorical_columns, initial_category, id='dropdown-category-selection')
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
        html.Div([
            html.Label("Select Values Within Category Column:"),
            dcc.Dropdown(id='dropdown-value-selection', multi=True, value=[])
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
        # html.Div([
        #     html.Label("Filter by value "),
        #     dcc.Dropdown(numerical_columns, numerical_columns[0], id='filter-selection'),
        #     dcc.Slider(id='value-slider', step=1)
        # ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
         html.Div([
            html.Label("X-Axis"),
            dcc.Dropdown(numerical_columns, numerical_columns[0], id='dropdown-x-selection')
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),  
        html.Div([
            html.Label("Y-Axis:"),
            dcc.Dropdown(numerical_columns, numerical_columns[1], id='dropdown-y-selection')
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
        html.Div([
            dcc.Checklist(options=[{'label': 'Include Regression Line', 'value': 'show'}], id='regression-toggle',value=[])  # Empty list means it's unchecked by default
        ], style={'marginTop': '20px', 'background-color': 'coral'}),
        dcc.Graph(id='graph-content')
    ])
    # # Callback to update the slider based on the selected category
    # @callback(
    #     Output('value-slider', 'min'),
    #     Output('value-slider', 'max'),
    #     Output('value-slider', 'marks'),
    #     Output('value-slider', 'value'),
    #     Input('filter-selection', 'value'),
    #     State('stored-data', 'data')
    # )
    # def update_slider(selected_category, stored_data):
    #     df = h.pd.DataFrame(stored_data)
    #     # Get min and max based on the selected category
    #     min_value = df[selected_category].min()
    #     max_value = df[selected_category].max()
    #     marks = {i: str(i) for i in range(int(min_value), int(max_value) + 1, (max_value - min_value) // 5 or 1)}  # Dynamically create marks with step based on range
    #     return min_value, max_value, marks, min_value  # Default to the min value
    
    # @callback(
    #     Output('stored-data', 'data'), 
    #     Input('filter-selection', 'value'),
    #     Input('value-slider', 'value'),
    #     State('stored-data', 'data')
    # )
    # def filter_by_numerical_category(filter_category, slider_value, stored_data):
    #     df = h.pd.DataFrame(stored_data)
    #     filtered_data = df[df[filter_category] >= slider_value]
    #     return filtered_data.to_dict()

    # Update the values dropdown based on the selected categorical column
    @callback(
        Output('dropdown-value-selection', 'options'),
        Output('dropdown-value-selection', 'value'),  # Output the random value
        Input('dropdown-category-selection', 'value')
       # State('stored-data', 'data'),
        # State('previous-selected-values', 'data'),  # Get the current options
       # State('previous-selected-category', 'data')  # Get the current options
    )
    def update_value_options(selected_category):#, stored_data, previous_options, previous_category):
        #df = h.pd.DataFrame(stored_data)
        # Get the unique values of the selected categorical column
        
       # if selected_category == previous_category:
      #      return previous_options
        
       # else:
        if(using_index):
            return 0
        else:
            # previous_category = selected_category
            unique_values = df[selected_category].unique()
            formatted_values = [{'label': str(i), 'value': str(i)} for i in unique_values]
            options = random.sample([i['value'] for i in formatted_values], 3)
            return formatted_values, options
    
    #update graph values
    @callback(
        Output('graph-content', 'figure'),
        Input('dropdown-category-selection', 'value'),
        Input('dropdown-value-selection', 'value'),
        Input('dropdown-x-selection', 'value'),
        Input('dropdown-y-selection', 'value'),
        Input('regression-toggle', 'value')
        #State('stored-data', 'data')
    )
    def update_graph(category_column, selected_values, valuex, valuey, regression_toggle): #, stored_data):
        #df = h.pd.DataFrame(stored_data)

        if selected_values:
            dff = data[data[category_column].isin(selected_values)]
            
        trendline = "ols" if "show" in regression_toggle else None  #sometimes it takes a little time for the regression line to appear on first loading up the notebook
        return px.scatter(dff, valuex, valuey, color=category_column, trendline=trendline, trendline_color_override='red')
    return app.layout

In [44]:
#this code block uses the dash module to visualize some of the data
df = h.pd.read_csv('dataset.csv')
df = df.drop(columns=['Unnamed: 0', 'track_id']) #remove index column

app = Dash()

app.layout = compare_axes(df, app)

if __name__ == '__main__':
    app.run(debug=True)


---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
Cell In[43], line 89, in compare_axes.<locals>.update_value_options(selected_category='artists')
     77 @callback(
     78     Output('dropdown-value-selection', 'options'),
     79     Output('dropdown-value-selection', 'value'),  # Output the random value
   (...)
     86     #df = h.pd.DataFrame(stored_data)
     87     # Get the unique values of the selected categorical column
---> 89     if selected_category == previous_category:
        selected_category = 'artists'
     90         return previous_options
     92     else:

UnboundLocalError: cannot access local variable 'previous_category' where it is not associated with a value

---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
Cell In[43], line 119, in compare_axes.<locals>.up